In [1]:
import cv2 as cv
import mediapipe as mp
import numpy as np
from time import time
import matplotlib.pyplot as plt
import math

Basis for idea:
https://bleedai.com/introduction-to-pose-detection-and-basic-pose-classification/

In [2]:

# Initializing mediapipe pose class.
mp_pose = mp.solutions.pose

In [3]:
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.4, model_complexity=1)

## min_detection_confidence = 0.5

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
# Initializing mediapipe drawing class, useful for annotation.
mp_drawing = mp.solutions.drawing_utils

In [11]:
def detectPose(image, pose, display=True):
    '''
    This function performs pose detection on an image.
    Args:
        image: The input image with a prominent person whose pose landmarks needs to be detected.
        pose: The pose setup function required to perform the pose detection.
        display: A boolean value that is if set to true the function displays the original input image, the resultant image, 
                 and the pose landmarks in 3D plot and returns nothing.
    Returns:
        output_image: The input image with the detected pose landmarks drawn.
        landmarks: A list of detected landmarks converted into their original scale.
    '''
    
    # Create a copy of the input image.
    output_image = image.copy()
    
    # Convert the image from BGR into RGB format.
    imageRGB = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    
    # Perform the Pose Detection.
    results = pose.process(imageRGB)
    
    # Retrieve the height and width of the input image.
    height, width, _ = image.shape
    
    # Initialize a list to store the detected landmarks.
    landmarks = []
    #prev_lm = []
    
    # Check if any landmarks are detected.
    if results.pose_landmarks:
    
        # Draw Pose landmarks on the output image.
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS)
        
        # Iterate over the detected landmarks.
        for landmark in results.pose_landmarks.landmark:
            
            # Append the landmark into the list.
            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                                  (landmark.z * width)))
            
            #prev_lm.append((int(landmark.x.shift(1).astype(int) * width), int(landmark.y.shift(1).astype(int) * height),
            #                (landmark.z.shift(1).astype(int) * width)))
    
    # Check if the original input image and the resultant image are specified to be displayed.
    if display:
    
        # Display the original input image and the resultant image.
        plt.figure(figsize=[22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        
        # Also Plot the Pose landmarks in 3D.
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
        
    # Otherwise
    else:
        
        # Return the output image and the found landmarks.
        return output_image, landmarks#, prev_lm

In [13]:
def calculateAngle(landmark1, landmark2, landmark3):
  

    # Get the required landmarks coordinates.
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    # Calculate the angle between the three points
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
  
    # Check if the angle is less than zero.
    if angle < 0:

        # Add 360 to the found angle.
        angle += 360
    

             
    
    # Return the calculated angle.
    return angle

Create definition returning x & shift(1) position of landmark
Create a separate definition calculating speed

In [14]:
def classifyPose(landmarks, frame, display=False):
    '''
    This function classifies QB throwing motion.
 
    '''
    
    # Initialize the label of the pose. It is not known at this stage.
    label = ' '
    re = ''
 
    # Specify the color (Red) with which the label will be written on the image.
    color = (255, 255, 255)
    
    # Calculate the required angles.
    #----------------------------------------------------------------------------------------------------------------
    
    # Get the angle between the left shoulder, elbow and wrist points. 
    left_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
    
    # Get the angle between the right shoulder, elbow and wrist points. 
    right_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])   
    
    # Get the angle between the left elbow, shoulder and hip points. 
    left_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])
 
    # Get the angle between the right hip, shoulder and elbow points. 
    right_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])
 
    # Get the angle between the left hip, knee and ankle points. 
    left_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
 
    # Get the angle between the right hip, knee and ankle points 
    right_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    
    
    
     #----------------------------------------------------------------------------------------------------------------
    # Find throw
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if the both arms are straight.
    if right_elbow_angle >= 45 and right_elbow_angle <= 90:     ## could likely tighthen this up
    
        label = "Throw"
        re = str("{:.2f}".format(right_elbow_angle))  
 
    if right_elbow_angle >= 90 and right_elbow_angle <= 180:
        
        label = "Windup/Ball Held"
        re = str("{:.2f}".format(right_elbow_angle))
        
    if right_elbow_angle > 180:    ### arm dropping 
        label = "Release"
        re = str("{:.2f}".format(right_elbow_angle))
  

    # Check if the pose is classified successfully
    if label != ' ':
        
        # Update the color (to green) with which the label will be written on the image.
        color = (255, 255, 255)  
    
    # Write the label on the output image. 
    cv.putText(frame, 'Throwing Classification: {}'.format(label), (10, 30),cv.FONT_HERSHEY_PLAIN, 2, color, 2)
    cv.putText(frame, 'Right Elbow Angle: {}'.format(re), (10, 60),cv.FONT_HERSHEY_PLAIN, 2, color, 2)
    
    # Check if the resultant image is specified to be displayed.
    if display:
    
        # Display the resultant image.
        plt.figure(figsize=[10,10])
        plt.imshow(frame[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        
    else:
        
        # Return the output image and the classified label.
        return frame, label

In [15]:
video = cv.VideoCapture('train/malik_trim_2.mp4')

In [16]:
# Initialize a variable to store the time of the previous frame.
time1 = 0

In [10]:
# Iterate until the video is accessed successfully.
while video.isOpened():
    
    # Read a frame.
    ok, frame = video.read()
    
    # Check if frame is not read properly.
    if not ok:
        
        # Break the loop.
        break

    
    # Get the width and height of the frame
    frame_height, frame_width, _ =  frame.shape
    
    # Resize the frame while keeping the aspect ratio.
    frame = cv.resize(frame, (int(frame_width * (640 / frame_height)), 640))
    
    # Perform Pose landmark detection.
    frame, landmarks  = detectPose(frame, pose_video, display=False)
    
    if landmarks:
    #ignore if no variables
        classifyPose(landmarks, frame, display=True)
        
    
    #if (landmarks - prev_lm) > 0:
    #    speed = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value] - prev_lm[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
    #    cv.putText(frame, 'Speed: {}'.format(int(speed)), (10, 120),cv.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)   ### calculate speed
    #    print(f'mph: {speed}')
        
    
    # Set the time for this frame to the current time.
    time2 = time()
    
    # Check if the difference between the previous and this frame time &gt; 0 to avoid division by zero.
    if (time2 - time1) >= 0:
    
        d = 1
        # Calculate the number of frames per second.
        frames_per_second = d / (time2 - time1)
        mph = 1 / 
        
        # Write the calculated number of frames per second on the frame. 
        cv.putText(frame, 'FPS: {}'.format(int(frames_per_second)), (10, 90),cv.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
        print(f'mph: {mph}')
        
    
    # Update the previous frame time to this frame time.
    # As this frame will become previous frame in next iteration.
    time1 = time2
    
    # Display the frame.
    cv.imshow('Pose Detection', frame)
    #cv.putText(right_elbow_angle,f'Right Elbow Angle: {right_elbow_angle}',(10,30),cv.FONT_HERSHEY_PLAIN,2,(0,255,0),2)

    
    # Wait until a key is pressed.
    # Retreive the ASCII code of the key pressed
    k = cv.waitKey(1) & 0xff
    
    # Check if 'ESC' is pressed.
    if(k == 27):
        
        # Break the loop.
        break

        
        
# Close the windows.
cv.destroyAllWindows()

AttributeError: 'float' object has no attribute 'shift'

Calculate angle of shoulder; create a flag if less than 45 degrees; time to 45 degrees? 

Calculate speed; distance; acceleration